In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas.tools import sjoin
path="C:/Users/52269198/OneDrive - Conduent/Documents/CONDUENT/ProjectBI/"
pd.set_option('display.max_columns',None)


from shapely import wkt

import os
from dateutil import relativedelta
import datetime


In [2]:
ny=pd.read_csv(path+"Motor_Vehicle_Collisions_-_Crashes_20231122.csv")

C:\Users\52269198\Anaconda3\envs\strong\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
ny['CRASH DATETIME'] = ny['CRASH DATE'] + ' ' + ny['CRASH TIME']

# Convert the combined column to datetime
ny['CRASH DATETIME'] = pd.to_datetime(ny['CRASH DATETIME'], format='%m/%d/%Y %H:%M',errors='coerce')
ny = ny.drop(['CRASH DATE', 'CRASH TIME'], axis=1)
ny = ny[['CRASH DATETIME'] + [col for col in ny.columns if col != 'CRASH DATETIME']]
ny

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,0.0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Pavement Slippery,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Turning Improperly,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2044337,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2044338,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2044339,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


In [12]:
ny.isna().sum()

CRASH DATETIME                         0
BOROUGH                           635972
ZIP CODE                          636217
LATITUDE                          231654
LONGITUDE                         231654
LOCATION                          231654
ON STREET NAME                    431992
CROSS STREET NAME                 768661
OFF STREET NAME                  1704722
NUMBER OF PERSONS INJURED             18
NUMBER OF PERSONS KILLED              31
NUMBER OF PEDESTRIANS INJURED          0
NUMBER OF PEDESTRIANS KILLED           0
NUMBER OF CYCLIST INJURED              0
NUMBER OF CYCLIST KILLED               0
NUMBER OF MOTORIST INJURED             0
NUMBER OF MOTORIST KILLED              0
CONTRIBUTING FACTOR VEHICLE 1       6538
CONTRIBUTING FACTOR VEHICLE 2     313889
CONTRIBUTING FACTOR VEHICLE 3    1898883
CONTRIBUTING FACTOR VEHICLE 4    2011638
CONTRIBUTING FACTOR VEHICLE 5    2035487
COLLISION_ID                           0
VEHICLE TYPE CODE 1                13113
VEHICLE TYPE COD

In [16]:
ny['CONTRIBUTING FACTOR VEHICLE 1'].value_counts(normalize=True).head(20)

Unspecified                       0.343055
Driver Inattention/Distraction    0.200084
Failure to Yield Right-of-Way     0.059381
Following Too Closely             0.053520
Backing Unsafely                  0.037189
Other Vehicular                   0.031147
Passing or Lane Usage Improper    0.027761
Passing Too Closely               0.024871
Turning Improperly                0.024803
Fatigued/Drowsy                   0.023246
Unsafe Lane Changing              0.019730
Traffic Control Disregarded       0.017673
Driver Inexperience               0.015640
Unsafe Speed                      0.013683
Alcohol Involvement               0.010853
Lost Consciousness                0.010021
Reaction to Uninvolved Vehicle    0.009713
Pavement Slippery                 0.009526
Prescription Medication           0.007563
View Obstructed/Limited           0.006595
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: float64

In [26]:
ny['CONTRIBUTING FACTOR VEHICLE 2'].value_counts(normalize=True).head(20)

Unspecified                       0.841927
Driver Inattention/Distraction    0.053705
Other Vehicular                   0.018307
Following Too Closely             0.010796
Failure to Yield Right-of-Way     0.009773
Passing or Lane Usage Improper    0.007307
Fatigued/Drowsy                   0.006261
Turning Improperly                0.005013
Passing Too Closely               0.004961
Backing Unsafely                  0.004527
Traffic Control Disregarded       0.004195
Unsafe Lane Changing              0.003830
Driver Inexperience               0.003810
Lost Consciousness                0.003024
Unsafe Speed                      0.002973
Pavement Slippery                 0.002336
Reaction to Uninvolved Vehicle    0.001902
View Obstructed/Limited           0.001784
Prescription Medication           0.001746
Outside Car Distraction           0.001482
Name: CONTRIBUTING FACTOR VEHICLE 2, dtype: float64

In [18]:
cars=ny[['CONTRIBUTING FACTOR VEHICLE 1','CONTRIBUTING FACTOR VEHICLE 2','CONTRIBUTING FACTOR VEHICLE 3','CONTRIBUTING FACTOR VEHICLE 4','CONTRIBUTING FACTOR VEHICLE 5']]
ny['NUM_CARS_INVOLVED'] = cars.notnull().sum(axis=1)
ny

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,0.0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN,2
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Pavement Slippery,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,1
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Following Too Closely,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,2
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,0.0,0,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN,1
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,0.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Turning Improperly,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN,2
2044337,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,0.0,1,0,0,0,0,0,Unspecified,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN,1
2044338,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN,1
2044339,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN,2


In [20]:
ny['NUM_CARS_INVOLVED'].value_counts(normalize=True)

2    0.775308
1    0.150342
3    0.055155
4    0.011666
5    0.004331
0    0.003198
Name: NUM_CARS_INVOLVED, dtype: float64

In [21]:
ny.BOROUGH.value_counts(normalize=True,dropna=False)

NaN              0.311089
BROOKLYN         0.218819
QUEENS           0.184563
MANHATTAN        0.154888
BRONX            0.101760
STATEN ISLAND    0.028880
Name: BOROUGH, dtype: float64

In [27]:
ny['VEHICLE TYPE CODE 1'].value_counts()

Sedan                                  562367
Station Wagon/Sport Utility Vehicle    442566
PASSENGER VEHICLE                      416206
SPORT UTILITY / STATION WAGON          180291
Taxi                                    50645
                                        ...  
Van (                                       1
Barri                                       1
comm.                                       1
UNK BOX TR                                  1
Utility Ve                                  1
Name: VEHICLE TYPE CODE 1, Length: 1590, dtype: int64

In [6]:
numberscols=['NUMBER OF PERSONS INJURED','NUMBER OF PERSONS KILLED','NUMBER OF PEDESTRIANS INJURED',
             'NUMBER OF PEDESTRIANS KILLED','NUMBER OF CYCLIST INJURED',
             'NUMBER OF CYCLIST KILLED','NUMBER OF MOTORIST INJURED','NUMBER OF MOTORIST KILLED']
pd.set_option('display.float_format', '{:.6f}'.format)

for e in numberscols:
    print(ny[e].describe())
    print()

count   1808390.000000
mean          0.303197
std           0.691488
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          43.000000
Name: NUMBER OF PERSONS INJURED, dtype: float64

count   1808390.000000
mean          0.001416
std           0.039792
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           8.000000
Name: NUMBER OF PERSONS KILLED, dtype: float64

count   1808390.000000
mean          0.058313
std           0.247713
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          27.000000
Name: NUMBER OF PEDESTRIANS INJURED, dtype: float64

count   1808390.000000
mean          0.000731
std           0.027535
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           6.000000
Name: NUMBER OF PEDESTRIANS KILLED, dtype: float64

count   1808390.000000
mean          0.027816
std           0.

In [41]:
ny[ny['NUM_CARS_INVOLVED']==2]['NUMBER OF PERSONS INJURED'].describe()

count   1584990.000000
mean          0.239773
std           0.637737
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          43.000000
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [4]:
ny.fillna(0,inplace=True)

In [6]:
ny[(ny['NUM_CARS_INVOLVED']==3)&(ny.BOROUGH=='BROOKLYN')]['NUMBER OF PERSONS INJURED'].describe()

count    23446.000000
mean         0.580568
std          1.060935
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         24.000000
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [11]:
print(numberscols)

['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED']


In [57]:
ny['NUMBER OF PEDESTRIANS INJURED %']=ny['NUMBER OF PEDESTRIANS INJURED']/ny['NUMBER OF PERSONS INJURED']
ny['NUMBER OF CYCLIST INJURED %']=ny['NUMBER OF CYCLIST INJURED']/ny['NUMBER OF PERSONS INJURED']
ny['NUMBER OF MOTORIST INJURED %']=ny['NUMBER OF MOTORIST INJURED']/ny['NUMBER OF PERSONS INJURED']
ny['NUMBER OF PEDESTRIANS KILLED %']=ny['NUMBER OF PEDESTRIANS KILLED']/ny['NUMBER OF PERSONS KILLED']
ny['NUMBER OF CYCLIST KILLED %']=ny['NUMBER OF CYCLIST KILLED']/ny['NUMBER OF PERSONS KILLED']
ny['NUMBER OF MOTORIST KILLED %']=ny['NUMBER OF MOTORIST KILLED']/ny['NUMBER OF PERSONS KILLED']

In [55]:
injured = ['NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED']

# Find the column name with the maximum value within the subset for each row
ny['max_injured'] = ny[injured].idxmax(axis=1)
ny[['NUMBER OF PERSONS INJURED','NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED','max_injured']]

,NUMBER OF PERSONS INJURED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF CYCLIST INJURED,NUMBER OF MOTORIST INJURED,max_injured
0,2,0,0,2,NUMBER OF MOTORIST INJURED
1,1,0,0,1,NUMBER OF MOTORIST INJURED
2,0,0,0,0,NUMBER OF PEDESTRIANS INJURED
3,0,0,0,0,NUMBER OF PEDESTRIANS INJURED
4,0,0,0,0,NUMBER OF PEDESTRIANS INJURED
...,...,...,...,...,...
2044336,0,0,0,0,NUMBER OF PEDESTRIANS INJURED
2044337,1,1,0,0,NUMBER OF PEDESTRIANS INJURED
2044338,0,0,0,0,NUMBER OF PEDESTRIANS INJURED
2044339,0,0,0,0,NUMBER OF PEDESTRIANS INJURED


In [58]:
a="NUMBER OF PEDESTRIANS INJURED"
b="NUMBER OF CYCLIST INJURED"
c="NUMBER OF MOTORIST INJURED"

ny.loc[((ny[a] == ny[b]) & (ny[a] > ny[c])) |
       ((ny[b] == ny[c]) & (ny[b] > ny[a])) |
       ((ny[a] == ny[c]) & (ny[a] > ny[b])) |
       ((ny[a] == ny[b]) & (ny[b] == ny[c])), 'max_injured'] = 'tie'
ny[['NUMBER OF PERSONS INJURED','NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED','max_injured']]

,NUMBER OF PERSONS INJURED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF CYCLIST INJURED,NUMBER OF MOTORIST INJURED,max_injured
0,2,0,0,2,NUMBER OF MOTORIST INJURED
1,1,0,0,1,NUMBER OF MOTORIST INJURED
2,0,0,0,0,tie
3,0,0,0,0,tie
4,0,0,0,0,tie
...,...,...,...,...,...
2044336,0,0,0,0,tie
2044337,1,1,0,0,NUMBER OF PEDESTRIANS INJURED
2044338,0,0,0,0,tie
2044339,0,0,0,0,tie


In [60]:
ny.loc[ny.max_injured.str.contains('PEDESTRIAN'),'max_injured']='Pedestrian'
ny.loc[ny.max_injured.str.contains('CYCLIST'),'max_injured']='Cyclist'
ny.loc[ny.max_injured.str.contains('MOTORIST'),'max_injured']='Motorist'
ny.loc[ny['NUMBER OF PERSONS INJURED']==0,'max_injured']=np.nan
ny.max_injured.value_counts(normalize=True,dropna=False)

NaN          0.777499
Motorist     0.143241
Pedestrian   0.052473
Cyclist      0.025862
tie          0.000925
Name: max_injured, dtype: float64

In [ ]:
########################33

In [61]:
killed = ['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED']

# Find the column name with the maximum value within the subset for each row
ny['max_killed'] = ny[killed].idxmax(axis=1)


In [62]:
a="NUMBER OF PEDESTRIANS KILLED"
b="NUMBER OF CYCLIST KILLED"
c="NUMBER OF MOTORIST KILLED"

ny.loc[((ny[a] == ny[b]) & (ny[a] > ny[c])) |
       ((ny[b] == ny[c]) & (ny[b] > ny[a])) |
       ((ny[a] == ny[c]) & (ny[a] > ny[b])) |
       ((ny[a] == ny[b]) & (ny[b] == ny[c])), 'max_killed'] = 'tie'
ny

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,NUMBER OF OTHER INJURED,weekday_name,hour,shift,weekday_shift,week,week_shift,CONTRIBUTING FACTOR VEHICLE 1st
0,2021-09-11 02:39:00,0,0.000000,0.000000,0.000000,0,WHITESTONE EXPRESSWAY,20 AVENUE,0,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,0,0,0,4455765,Sedan,Sedan,0,0,0,2,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,tie,0.000000,Saturday,2,night,Saturday night,weekend,weekend night,Others
1,2022-03-26 11:45:00,0,0.000000,0.000000,0.000000,0,QUEENSBORO BRIDGE UPPER,0,0,1,0,0,0,0,0,1,0,Pavement Slippery,0,0,0,0,4513547,Sedan,0,0,0,0,1,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,tie,0.000000,Saturday,11,morning,Saturday morning,weekend,weekend morning,Others
2,2022-06-29 06:55:00,0,0.000000,0.000000,0.000000,0,THROGS NECK BRIDGE,0,0,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,0,0,0,4541903,Sedan,Pick-up Truck,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,0.000000,Wednesday,6,earlymorning,Wednesday earlymorning,week,week earlymorning,Following Too Closely
3,2021-09-11 09:35:00,BROOKLYN,11208.000000,40.667202,-73.866500,"(40.667202, -73.8665)",0,0,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,0,0,0,0,4456314,Sedan,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,0.000000,Saturday,9,morning,Saturday morning,weekend,weekend morning,Unspecified
4,2021-12-14 08:13:00,BROOKLYN,11233.000000,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4486609,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,0.000000,Tuesday,8,morning,Tuesday morning,week,week morning,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,0,0.000000,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,0,0,0,0,0,0,0,0,0,0,Turning Improperly,Unspecified,0,0,0,4648110,Sedan,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,0.000000,Monday,18,afternoon,Monday afternoon,week,week afternoon,Turning Improperly
2044337,2023-07-22 21:39:00,BRONX,10457.000000,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,0,1,0,1,0,0,0,0,0,Unspecified,0,0,0,0,4648117,Sedan,0,0,0,0,1,1.000000,0.000000,0.000000,NaN,NaN,NaN,Pedestrian,tie,0.000000,Saturday,21,night,Saturday night,weekend,weekend night,Unspecified
2044338,2023-07-02 17:55:00,MANHATTAN,10006.000000,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4648366,Taxi,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,0.000000,Sunday,17,afternoon,Sunday afternoon,weekend,weekend afternoon,Driver Inattention/Distraction
2044339,2023-07-22 13:15:00,QUEENS,11433.000000,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tie,1.000000,Saturday,13,noon,Saturday noon,weekend,weekend noon,Driver Inattention/Distraction


In [63]:
ny.loc[ny.max_killed.str.contains('PEDESTRIAN'),'max_killed']='Pedestrian'
ny.loc[ny.max_killed.str.contains('CYCLIST'),'max_killed']='Cyclist'
ny.loc[ny.max_killed.str.contains('MOTORIST'),'max_killed']='Motorist'
ny.loc[ny['NUMBER OF PERSONS KILLED']==0,'max_killed']=np.nan
ny.max_killed.value_counts(normalize=True,dropna=False)

NaN          0.998610
Pedestrian   0.000721
Motorist     0.000554
Cyclist      0.000112
tie          0.000003
Name: max_killed, dtype: float64

In [33]:
ny.max_injured.value_counts(dropna=False)

NaN           1583031
tie            406621
Cyclist         52865
Pedestrian       1324
Motorist          500
Name: max_injured, dtype: int64

In [29]:
ny.max_killed.value_counts(dropna=False)

NaN           2041454
tie              2652
Cyclist           228
Pedestrian          7
Name: max_killed, dtype: int64

In [ ]:
######################3

In [64]:
ny.max_injured.value_counts(dropna=False)

NaN           1589474
Motorist       292833
Pedestrian     107272
Cyclist         52870
tie              1892
Name: max_injured, dtype: int64

In [65]:
ny.max_killed.value_counts(dropna=False)

NaN           2041500
Pedestrian       1474
Motorist         1132
Cyclist           228
tie                 7
Name: max_killed, dtype: int64

In [75]:
ny[['NUMBER OF PERSONS INJURED','NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF CYCLIST INJURED', 'NUMBER OF MOTORIST INJURED','max_injured']].sample(10)

,NUMBER OF PERSONS INJURED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF CYCLIST INJURED,NUMBER OF MOTORIST INJURED,max_injured
792989,0,0,0,0,NaN
907503,1,0,1,0,Cyclist
933821,0,0,0,0,NaN
1036052,1,1,0,0,Pedestrian
1191935,0,0,0,0,NaN
2044313,0,0,0,0,NaN
1406555,0,0,0,0,NaN
2036661,3,0,0,3,Motorist
1955722,0,0,0,0,NaN
1534864,0,0,0,0,NaN


In [72]:
ny[ny['NUMBER OF PERSONS KILLED']>0][['NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST KILLED','max_killed']].sample(10)

,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST KILLED,max_killed
1602613,1,0,0,Pedestrian
1194683,0,0,1,Motorist
1746749,1,0,0,Pedestrian
1958083,1,0,0,Pedestrian
1071667,1,0,0,Pedestrian
1030066,1,0,0,Pedestrian
859393,1,0,0,Pedestrian
490632,1,0,0,Pedestrian
1319758,0,0,1,Motorist
2018213,0,0,1,Motorist


In [5]:
print(len(ny[ny.BOROUGH=='BROOKLYN']))
ny[ny.BOROUGH=='BROOKLYN']['NUMBER OF PERSONS INJURED'].sum()

447341


146443.0

In [39]:
len(ny[(ny.BOROUGH=='BROOKLYN')&(~ny['NUMBER OF PEDESTRIANS INJURED'].isna())])

447341

In [40]:
ny[ny.max_killed=='tie'] #calcular de nuevo

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed
39,2021-07-09 00:43:00,0,0,40.720535,-73.888850,"(40.720535, -73.88885)",ELIOT AVENUE,0,0,0.000000,1.000000,0,1,0,0,0,0,Unspecified,0,0,0,0,4456659,Bus,0,0,0,0,1,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,tie
148,2021-12-12 09:09:00,0,0,40.840360,-73.918070,"(40.84036, -73.91807)",JEROME AVENUE,0,0,0.000000,1.000000,0,1,0,0,0,0,Unspecified,0,0,0,0,4487210,Taxi,0,0,0,0,1,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,tie
591,2021-04-15 15:18:00,BROOKLYN,11209.000000,40.620487,-74.029305,"(40.620487, -74.029305)",4 AVENUE,FOREST PLACE,0,0.000000,1.000000,0,1,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4408063,Station Wagon/Sport Utility Vehicle,0,0,0,0,1,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,tie
605,2021-04-15 22:36:00,0,0,0.000000,0.000000,0,Trans- Manhattan Expressway,Amsterdam Avenue,0,4.000000,1.000000,0,0,0,0,4,1,Alcohol Involvement,0,0,0,0,4407693,Sedan,0,0,0,0,1,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,tie,tie
1320,2021-04-17 13:31:00,0,0,40.782463,-73.978830,"(40.782463, -73.97883)",AMSTERDAM AVENUE,0,0,0.000000,1.000000,0,1,0,0,0,0,Unsafe Speed,0,0,0,0,4408062,E-Bike,0,0,0,0,1,NaN,NaN,NaN,1.000000,0.000000,0.000000,NaN,tie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042723,2023-07-01 00:27:00,QUEENS,11372.000000,40.753536,-73.886900,"(40.753536, -73.8869)",80 STREET,34 AVENUE,0,3.000000,1.000000,0,0,0,0,3,1,Traffic Control Disregarded,Unspecified,0,0,0,4643896,Motorcycle,0,0,0,0,2,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,tie,tie
2042724,2023-07-09 09:25:00,QUEENS,11103.000000,40.764730,-73.912110,"(40.76473, -73.91211)",28 AVENUE,42 STREET,0,0.000000,1.000000,0,0,0,0,0,1,Traffic Control Disregarded,Unspecified,0,0,0,4643897,Station Wagon/Sport Utility Vehicle,Moped,0,0,0,2,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,tie
2042872,2023-07-17 22:15:00,BROOKLYN,11236.000000,40.632435,-73.888180,"(40.632435, -73.88818)",ROCKAWAY PARKWAY,SKIDMORE AVENUE,0,0.000000,1.000000,0,0,0,0,0,1,Failure to Yield Right-of-Way,Unspecified,0,0,0,4646703,Sedan,Motorcycle,0,0,0,2,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,tie
2044140,2023-07-22 11:17:00,QUEENS,11429.000000,40.705220,-73.727880,"(40.70522, -73.72788)",112 AVENUE,CROSS ISLAND PARKWAY,0,0.000000,1.000000,0,0,0,0,0,1,Unsafe Speed,0,0,0,0,4648067,Station Wagon/Sport Utility Vehicle,0,0,0,0,1,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,tie


In [12]:
ny['injured sum']=ny['NUMBER OF PEDESTRIANS INJURED']+ny['NUMBER OF MOTORIST INJURED']+ny['NUMBER OF CYCLIST INJURED']
ny[ny['injured sum']!=ny['NUMBER OF PERSONS INJURED']]

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,injured sum
47,2022-04-15 06:11:00,0,0.0,0.000000,0.000000,"(0.0, 0.0)",EAST 162 STREET,0,0,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,0,0,0,4522124,Station Wagon/Sport Utility Vehicle,E-Scooter,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
284,2021-09-11 13:00:00,0,0.0,40.680477,-73.951164,"(40.680477, -73.951164)",FULTON STREET,0,0,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4456154,Sedan,E-Scooter,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
356,2021-07-07 17:13:00,QUEENS,11433.0,40.688435,-73.787650,"(40.688435, -73.78765)",LINDEN BOULEVARD,BEDELL STREET,0,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,0,0,0,4456609,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
385,2021-09-10 21:03:00,MANHATTAN,10002.0,40.710697,-73.984634,"(40.710697, -73.984634)",0,0,299 SOUTH STREET,1.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,0,0,0,4456953,Sedan,E-Scooter,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
409,2021-09-10 17:39:00,BRONX,10467.0,40.877390,-73.866500,"(40.87739, -73.8665)",WHITE PLAINS ROAD,EAST GUN HILL ROAD,0,1.0,0.0,0,0,0,0,0,0,Unsafe Lane Changing,Driver Inattention/Distraction,0,0,0,4457006,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044199,2023-07-21 14:25:00,BROOKLYN,11218.0,40.638523,-73.973190,"(40.638523, -73.97319)",CORTELYOU ROAD,OCEAN PARKWAY,0,1.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,0,0,0,4648009,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
2044255,2023-07-22 14:12:00,0,0.0,40.877460,-73.853745,"(40.87746, -73.853745)",LACONIA AVENUE,0,0,1.0,0.0,0,0,0,0,0,0,Unsafe Speed,Unspecified,0,0,0,4647836,Sedan,E-Bike,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
2044286,2023-07-22 16:33:00,0,0.0,0.000000,0.000000,0,EAST BROADWAY,ESSEX STREET,0,1.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648246,E-Bike,Station Wagon/Sport Utility Vehicle,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0
2044313,2023-07-22 09:00:00,0,0.0,40.742160,-73.804436,"(40.74216, -73.804436)",164 STREET,0,0,2.0,0.0,0,0,0,0,0,0,Passing or Lane Usage Improper,Unsafe Speed,0,0,0,4647882,E-Scooter,E-Scooter,0,0,0,2,0.0,0.0,0.0,NaN,NaN,NaN,tie,NaN,0


In [13]:
ny['killed sum']=ny['NUMBER OF PEDESTRIANS KILLED']+ny['NUMBER OF MOTORIST KILLED']+ny['NUMBER OF CYCLIST KILLED']
ny[ny['killed sum']!=ny['NUMBER OF PERSONS KILLED']]

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,injured sum,killed sum
2607,2021-04-08 19:55:00,BRONX,10459.0,40.830307,-73.898730,"(40.830307, -73.89873)",0,0,1281 UNION AVENUE,0.0,1.0,0,0,0,0,0,0,Driver Inexperience,0,0,0,0,4407509,E-Bike,0,0,0,0,1,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
5831,2021-04-24 03:43:00,BROOKLYN,11226.0,40.643063,-73.951660,"(40.643063, -73.95166)",ROGERS AVENUE,CLARENDON ROAD,0,0.0,1.0,0,0,0,0,0,0,Unspecified,Unspecified,0,0,0,4410388,Sedan,E-Bike,0,0,0,2,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
19612,2021-05-06 14:02:00,STATEN ISLAND,10307.0,40.505527,-74.238190,"(40.505527, -74.23819)",HYLAN BOULEVARD,SPRAGUE AVENUE,0,0.0,1.0,0,0,0,0,0,0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Unspecified,0,0,0,4415906,E-Bike,Pick-up Truck,0,0,0,2,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
24447,2022-06-30 14:29:00,BROOKLYN,11219.0,40.633083,-74.005580,"(40.633083, -74.00558)",FORT HAMILTON PARKWAY,60 STREET,0,0.0,1.0,0,0,0,0,0,0,Other Vehicular,Unspecified,Unspecified,0,0,4542311,Tractor Truck Diesel,Sedan,E-Scooter,0,0,3,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
29256,2021-10-02 02:46:00,BRONX,10457.0,40.844970,-73.902470,"(40.84497, -73.90247)",WEBSTER AVENUE,ITTNER PLACE,0,0.0,1.0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4463262,E-Bike,0,0,0,0,1,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
32260,2021-10-11 07:21:00,0,0.0,40.747060,-73.929344,"(40.74706, -73.929344)",HONEYWELL STREET,0,0,0.0,1.0,0,0,0,0,0,0,Unsafe Speed,Unspecified,0,0,0,4466536,Pick-up Truck,E-Bike,0,0,0,2,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
45129,2021-06-19 19:17:00,0,0.0,40.839676,-73.871216,"(40.839676, -73.871216)",0,0,EAST TREMONT AVENUE,0.0,1.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,Unspecified,0,0,4430829,Station Wagon/Sport Utility Vehicle,E-Scooter,Station Wagon/Sport Utility Vehicle,0,0,3,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
50664,2022-01-14 04:28:00,BRONX,10472.0,40.828660,-73.878240,"(40.82866, -73.87824)",WESTCHESTER AVENUE,BOYNTON AVENUE,0,0.0,1.0,0,0,0,0,0,0,Unspecified,0,0,0,0,4494526,E-Bike,0,0,0,0,1,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
51298,2021-05-28 20:00:00,0,0.0,40.820515,-73.930695,"(40.820515, -73.930695)",EXTERIOR STREET,0,0,0.0,1.0,0,0,0,0,0,0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,Traffic Control Disregarded,0,0,0,4421887,Sedan,E-Bike,0,0,0,2,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0
57167,2022-07-04 04:46:00,0,0.0,40.689102,-73.945070,"(40.689102, -73.94507)",GREENE AVENUE,0,0,0.0,1.0,0,0,0,0,0,0,Unspecified,Unspecified,0,0,0,4544251,E-Bike,Station Wagon/Sport Utility Vehicle,0,0,0,2,NaN,NaN,NaN,0.0,0.0,0.0,NaN,tie,0,0


In [18]:
ny['NUMBER OF PERSONS KILLED']=ny['killed sum']
ny['NUMBER OF PERSONS INJURED']=ny['injured sum']

In [19]:
ny.drop(['injured sum','killed sum'],axis=1,inplace=True)

In [17]:
ny[ny['NUMBER OF OTHER INJURED']<0]

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,injured sum,killed sum,NUMBER OF OTHER INJURED
182093,2021-01-28 10:10:00,0,0.000000,40.866600,-73.895454,"(40.8666, -73.895454)",EAST KINGSBRIDGE ROAD,0,0,0.000000,0.000000,0,0,1,0,0,0,Driver Inattention/Distraction,Unspecified,0,0,0,4387369,Station Wagon/Sport Utility Vehicle,Bike,0,0,0,2,NaN,inf,NaN,NaN,NaN,NaN,NaN,NaN,1,0,-1.000000
506915,2018-12-18 19:10:00,QUEENS,11373.000000,40.734440,-73.873250,"(40.73444, -73.87325)",0,0,89-03 57 AVENUE,0.000000,0.000000,0,0,0,0,1,0,Unspecified,0,0,0,0,4056103,Sedan,0,0,0,0,1,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,1,0,-1.000000
560518,2018-10-14 03:00:00,0,0.000000,40.742676,-73.839516,"(40.742676, -73.839516)",LONG ISLAND EXPRESSWAY,0,0,0.000000,0.000000,1,0,0,0,0,0,Unspecified,0,0,0,0,4032226,Sedan,0,0,0,0,1,inf,NaN,NaN,NaN,NaN,NaN,Pedestrian,NaN,1,0,-1.000000
565411,2018-10-01 11:20:00,QUEENS,11416.000000,40.687570,-73.848570,"(40.68757, -73.84857)",0,0,94-22 94 STREET,0.000000,0.000000,0,0,0,0,1,0,Backing Unsafely,Unspecified,0,0,0,4026379,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,0,0,0,2,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,1,0,-1.000000
566556,2018-10-04 13:53:00,MANHATTAN,10036.000000,40.759790,-73.987930,"(40.75979, -73.98793)",8 AVENUE,WEST 46 STREET,0,1.000000,0.000000,1,0,0,0,1,0,Unspecified,0,0,0,0,4026516,Sedan,0,0,0,0,1,1.000000,0.000000,1.000000,NaN,NaN,NaN,tie,NaN,2,0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1166801,2016-03-15 07:30:00,STATEN ISLAND,10305.000000,0.000000,0.000000,0,WEST FINGERBOARD ROAD,NORTH RAILROAD AVENUE,0,1.000000,0.000000,0,0,0,0,2,0,Illnes,Unspecified,0,0,0,3430386,4 dr sedan,4 dr sedan,0,0,0,2,0.000000,0.000000,2.000000,NaN,NaN,NaN,Motorist,NaN,2,0,-1.000000
1167713,2016-03-14 14:00:00,BROOKLYN,11214.000000,0.000000,0.000000,0,CROPSEY AVENUE,BAY 52 STREET,0,0.000000,0.000000,0,0,0,0,4,0,Turning Improperly,Unspecified,0,0,0,3414482,2 dr sedan,Station Wagon/Sport Utility Vehicle,0,0,0,2,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,4,0,-4.000000
1168095,2016-03-14 09:20:00,0,0.000000,0.000000,0.000000,0,0,WHITE PLAINS ROAD,0,0.000000,0.000000,0,0,0,0,3,0,Unspecified,Unspecified,0,0,0,4022111,Pick-up Truck,4 dr sedan,0,0,0,2,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,3,0,-3.000000
1168182,2016-03-15 21:17:00,0,0.000000,0.000000,0.000000,0,0,0,0,0.000000,0.000000,0,0,0,0,1,0,Driver Inattention/Distraction,0,0,0,0,3407999,4 dr sedan,4 dr sedan,0,0,0,1,NaN,NaN,inf,NaN,NaN,NaN,NaN,NaN,1,0,-1.000000


In [6]:
ny.isna().sum()

CRASH DATETIME                           0
BOROUGH                                  0
ZIP CODE                                 0
LATITUDE                                 0
LONGITUDE                                0
LOCATION                                 0
ON STREET NAME                           0
CROSS STREET NAME                        0
OFF STREET NAME                          0
NUMBER OF PERSONS INJURED                0
NUMBER OF PERSONS KILLED                 0
NUMBER OF PEDESTRIANS INJURED            0
NUMBER OF PEDESTRIANS KILLED             0
NUMBER OF CYCLIST INJURED                0
NUMBER OF CYCLIST KILLED                 0
NUMBER OF MOTORIST INJURED               0
NUMBER OF MOTORIST KILLED                0
CONTRIBUTING FACTOR VEHICLE 1            0
CONTRIBUTING FACTOR VEHICLE 2            0
CONTRIBUTING FACTOR VEHICLE 3            0
CONTRIBUTING FACTOR VEHICLE 4            0
CONTRIBUTING FACTOR VEHICLE 5            0
COLLISION_ID                             0
VEHICLE TYP

In [25]:
print(ny['ON STREET NAME'].nunique())
ny['CRASH DATETIME'] = pd.to_datetime(ny['CRASH DATETIME'],errors='coerce')
ny['weekday_name'] = ny['CRASH DATETIME'].dt.day_name()
ny['hour'] = ny['CRASH DATETIME'].dt.hour
ny

18234


,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,NUMBER OF OTHER INJURED,weekday_name,hour
0,2021-09-11 02:39:00,0,0.000000,0.000000,0.000000,0,WHITESTONE EXPRESSWAY,20 AVENUE,0,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,0,0,0,4455765,Sedan,Sedan,0,0,0,2,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,2
1,2022-03-26 11:45:00,0,0.000000,0.000000,0.000000,0,QUEENSBORO BRIDGE UPPER,0,0,1,0,0,0,0,0,1,0,Pavement Slippery,0,0,0,0,4513547,Sedan,0,0,0,0,1,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,11
2,2022-06-29 06:55:00,0,0.000000,0.000000,0.000000,0,THROGS NECK BRIDGE,0,0,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,0,0,0,4541903,Sedan,Pick-up Truck,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Wednesday,6
3,2021-09-11 09:35:00,BROOKLYN,11208.000000,40.667202,-73.866500,"(40.667202, -73.8665)",0,0,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,0,0,0,0,4456314,Sedan,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Saturday,9
4,2021-12-14 08:13:00,BROOKLYN,11233.000000,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4486609,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Tuesday,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,0,0.000000,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,0,0,0,0,0,0,0,0,0,0,Turning Improperly,Unspecified,0,0,0,4648110,Sedan,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Monday,18
2044337,2023-07-22 21:39:00,BRONX,10457.000000,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,0,1,0,1,0,0,0,0,0,Unspecified,0,0,0,0,4648117,Sedan,0,0,0,0,1,1.000000,0.000000,0.000000,NaN,NaN,NaN,Pedestrian,NaN,0.000000,Saturday,21
2044338,2023-07-02 17:55:00,MANHATTAN,10006.000000,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4648366,Taxi,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Sunday,17
2044339,2023-07-22 13:15:00,QUEENS,11433.000000,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.000000,0.000000,0.000000,NaN,NaN,NaN,tie,NaN,1.000000,Saturday,13


In [26]:
ny.weekday_name.value_counts(normalize=True)

Friday      0.159394
Thursday    0.149558
Tuesday     0.147247
Wednesday   0.146269
Monday      0.142810
Saturday    0.134759
Sunday      0.119963
Name: weekday_name, dtype: float64

In [29]:
morningshift=[8,9,10,11]
noonshift=[12,13,14]
afternoonshift=[15,16,17,18]
nightshift=[19,20,21,22,23,0,1,2,3]
earlymorningshift=[4,5,6,7]
ny['shift']=np.nan
ny.loc[ny.hour.isin(morningshift),'shift']='morning'
ny.loc[ny.hour.isin(noonshift),'shift']='noon'
ny.loc[ny.hour.isin(afternoonshift),'shift']='afternoon'
ny.loc[ny.hour.isin(nightshift),'shift']='night'
ny.loc[ny.hour.isin(earlymorningshift),'shift']='earlymorning'
ny['shift'].value_counts(normalize=True)

afternoon      0.267351
night          0.263776
morning        0.209341
noon           0.179561
earlymorning   0.079971
Name: shift, dtype: float64

In [34]:
ny['week']=np.nan
ny.loc[ny.weekday_name.isin(['Monday','Tuesday','Wednesday','Thursday']),'week']='week'
ny.loc[~ny.weekday_name.isin(['Monday','Tuesday','Wednesday','Thursday']),'week']='weekend'

In [35]:
pd.crosstab(ny.week,ny['shift'],normalize=True)

shift,afternoon,earlymorning,morning,night,noon
week,,,,,
week,0.161367,0.046244,0.137551,0.135004,0.105718
weekend,0.105984,0.033726,0.071790,0.128773,0.073843


In [33]:
ny['weekday_shift']=ny['weekday_name']+" "+ny['shift']
ny['weekday_shift'].value_counts(normalize=True)

Saturday night           0.045623
Friday afternoon         0.043628
Sunday night             0.041612
Friday night             0.041538
Wednesday afternoon      0.040898
Thursday afternoon       0.040895
Tuesday afternoon        0.040684
Monday afternoon         0.038890
Thursday night           0.036102
Tuesday morning          0.035611
Thursday morning         0.034515
Wednesday night          0.033926
Monday morning           0.033823
Friday morning           0.033760
Wednesday morning        0.033602
Saturday afternoon       0.032790
Tuesday night            0.032717
Monday night             0.032259
Sunday afternoon         0.029566
Friday noon              0.028409
Tuesday noon             0.026793
Thursday noon            0.026717
Wednesday noon           0.026444
Monday noon              0.025764
Saturday noon            0.024146
Saturday morning         0.021406
Sunday noon              0.021288
Sunday morning           0.016625
Monday earlymorning      0.012075
Friday earlymo

In [37]:
ny['week_shift']=ny['week']+" "+ny['shift']
ny['week_shift'].value_counts(normalize=True)

week afternoon         0.161367
week morning           0.137551
week night             0.135004
weekend night          0.128773
weekend afternoon      0.105984
week noon              0.105718
weekend noon           0.073843
weekend morning        0.071790
week earlymorning      0.046244
weekend earlymorning   0.033726
Name: week_shift, dtype: float64

In [42]:
ny[ny['CONTRIBUTING FACTOR VEHICLE 1']!='0']

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,NUMBER OF OTHER INJURED,weekday_name,hour,shift,weekday_shift,week,week_shift
0,2021-09-11 02:39:00,0,0.000000,0.000000,0.000000,0,WHITESTONE EXPRESSWAY,20 AVENUE,0,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,0,0,0,4455765,Sedan,Sedan,0,0,0,2,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,2,night,Saturday night,weekend,weekend night
1,2022-03-26 11:45:00,0,0.000000,0.000000,0.000000,0,QUEENSBORO BRIDGE UPPER,0,0,1,0,0,0,0,0,1,0,Pavement Slippery,0,0,0,0,4513547,Sedan,0,0,0,0,1,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,11,morning,Saturday morning,weekend,weekend morning
2,2022-06-29 06:55:00,0,0.000000,0.000000,0.000000,0,THROGS NECK BRIDGE,0,0,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,0,0,0,4541903,Sedan,Pick-up Truck,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Wednesday,6,earlymorning,Wednesday earlymorning,week,week earlymorning
3,2021-09-11 09:35:00,BROOKLYN,11208.000000,40.667202,-73.866500,"(40.667202, -73.8665)",0,0,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,0,0,0,0,4456314,Sedan,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Saturday,9,morning,Saturday morning,weekend,weekend morning
5,2021-04-14 12:47:00,0,0.000000,0.000000,0.000000,0,MAJOR DEEGAN EXPRESSWAY RAMP,0,0,0,0,0,0,0,0,0,0,Unspecified,Unspecified,0,0,0,4407458,Dump,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Wednesday,12,noon,Wednesday noon,week,week noon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,0,0.000000,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,0,0,0,0,0,0,0,0,0,0,Turning Improperly,Unspecified,0,0,0,4648110,Sedan,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Monday,18,afternoon,Monday afternoon,week,week afternoon
2044337,2023-07-22 21:39:00,BRONX,10457.000000,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,0,1,0,1,0,0,0,0,0,Unspecified,0,0,0,0,4648117,Sedan,0,0,0,0,1,1.000000,0.000000,0.000000,NaN,NaN,NaN,Pedestrian,NaN,0.000000,Saturday,21,night,Saturday night,weekend,weekend night
2044338,2023-07-02 17:55:00,MANHATTAN,10006.000000,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4648366,Taxi,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Sunday,17,afternoon,Sunday afternoon,weekend,weekend afternoon
2044339,2023-07-22 13:15:00,QUEENS,11433.000000,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.000000,0.000000,0.000000,NaN,NaN,NaN,tie,NaN,1.000000,Saturday,13,noon,Saturday noon,weekend,weekend noon


In [48]:
factorstop=list(ny['CONTRIBUTING FACTOR VEHICLE 1'].value_counts(normalize=True).head(10).index)

In [49]:
ny['CONTRIBUTING FACTOR VEHICLE 1st']=ny['CONTRIBUTING FACTOR VEHICLE 1']
ny.loc[~ny['CONTRIBUTING FACTOR VEHICLE 1'].isin(factorstop),'CONTRIBUTING FACTOR VEHICLE 1st']='Others'
ny['CONTRIBUTING FACTOR VEHICLE 1st'].value_counts(normalize=True)

Unspecified                      0.341958
Driver Inattention/Distraction   0.199444
Others                           0.177582
Failure to Yield Right-of-Way    0.059191
Following Too Closely            0.053349
Backing Unsafely                 0.037070
Other Vehicular                  0.031047
Passing or Lane Usage Improper   0.027672
Passing Too Closely              0.024792
Turning Improperly               0.024724
Fatigued/Drowsy                  0.023171
Name: CONTRIBUTING FACTOR VEHICLE 1st, dtype: float64

In [51]:
ny['CONTRIBUTING FACTOR VEHICLE 2'].value_counts(normalize=True)

Unspecified                      0.712657
0                                0.153540
Driver Inattention/Distraction   0.045459
Other Vehicular                  0.015496
Following Too Closely            0.009138
                                   ...   
Eating or Drinking               0.000005
Listening/Using Headphones       0.000005
Windshield Inadequate            0.000003
1                                0.000002
Texting                          0.000001
Name: CONTRIBUTING FACTOR VEHICLE 2, Length: 62, dtype: float64

In [52]:
#calles mas injuries por rate y todo eso etc
ny

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,NUMBER OF OTHER INJURED,weekday_name,hour,shift,weekday_shift,week,week_shift,CONTRIBUTING FACTOR VEHICLE 1st
0,2021-09-11 02:39:00,0,0.000000,0.000000,0.000000,0,WHITESTONE EXPRESSWAY,20 AVENUE,0,2,0,0,0,0,0,2,0,Aggressive Driving/Road Rage,Unspecified,0,0,0,4455765,Sedan,Sedan,0,0,0,2,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,2,night,Saturday night,weekend,weekend night,Others
1,2022-03-26 11:45:00,0,0.000000,0.000000,0.000000,0,QUEENSBORO BRIDGE UPPER,0,0,1,0,0,0,0,0,1,0,Pavement Slippery,0,0,0,0,4513547,Sedan,0,0,0,0,1,0.000000,0.000000,1.000000,NaN,NaN,NaN,Motorist,NaN,0.000000,Saturday,11,morning,Saturday morning,weekend,weekend morning,Others
2,2022-06-29 06:55:00,0,0.000000,0.000000,0.000000,0,THROGS NECK BRIDGE,0,0,0,0,0,0,0,0,0,0,Following Too Closely,Unspecified,0,0,0,4541903,Sedan,Pick-up Truck,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Wednesday,6,earlymorning,Wednesday earlymorning,week,week earlymorning,Following Too Closely
3,2021-09-11 09:35:00,BROOKLYN,11208.000000,40.667202,-73.866500,"(40.667202, -73.8665)",0,0,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,0,0,0,0,4456314,Sedan,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Saturday,9,morning,Saturday morning,weekend,weekend morning,Unspecified
4,2021-12-14 08:13:00,BROOKLYN,11233.000000,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4486609,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Tuesday,8,morning,Tuesday morning,week,week morning,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,0,0.000000,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,0,0,0,0,0,0,0,0,0,0,Turning Improperly,Unspecified,0,0,0,4648110,Sedan,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Monday,18,afternoon,Monday afternoon,week,week afternoon,Turning Improperly
2044337,2023-07-22 21:39:00,BRONX,10457.000000,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,0,1,0,1,0,0,0,0,0,Unspecified,0,0,0,0,4648117,Sedan,0,0,0,0,1,1.000000,0.000000,0.000000,NaN,NaN,NaN,Pedestrian,NaN,0.000000,Saturday,21,night,Saturday night,weekend,weekend night,Unspecified
2044338,2023-07-02 17:55:00,MANHATTAN,10006.000000,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4648366,Taxi,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,Sunday,17,afternoon,Sunday afternoon,weekend,weekend afternoon,Driver Inattention/Distraction
2044339,2023-07-22 13:15:00,QUEENS,11433.000000,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,0.000000,0.000000,0.000000,NaN,NaN,NaN,tie,NaN,1.000000,Saturday,13,noon,Saturday noon,weekend,weekend noon,Driver Inattention/Distraction


In [ ]:
#CONTRIBUTING FACTOR VEHICLE 1st
#NUM_CARS_INVOLVED
#week_shift


In [4]:
ny.drop('LOCATION',axis=1,inplace=True)
ny=ny[ny.LATITUDE!=0]
ny

,CRASH DATETIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,NUM_CARS_INVOLVED,NUMBER OF PEDESTRIANS INJURED %,NUMBER OF CYCLIST INJURED %,NUMBER OF MOTORIST INJURED %,NUMBER OF PEDESTRIANS KILLED %,NUMBER OF CYCLIST KILLED %,NUMBER OF MOTORIST KILLED %,max_injured,max_killed,NUMBER OF OTHER INJURED,weekday_name,hour,shift,weekday_shift,week,week_shift,CONTRIBUTING FACTOR VEHICLE 1st
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,0,0,1211 LORING AVENUE,0,0,0,0,0,0,0,0,Unspecified,0,0,0,0,4456314,Sedan,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Saturday,9,morning,Saturday morning,weekend,weekend morning,Unspecified
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,SARATOGA AVENUE,DECATUR STREET,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4486609,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tuesday,8,morning,Tuesday morning,week,week morning,Others
6,2021-12-14 17:05:00,0,0.0,40.709183,-73.956825,BROOKLYN QUEENS EXPRESSWAY,0,0,0,0,0,0,0,0,0,0,Passing Too Closely,Unspecified,0,0,0,4486555,Sedan,Tractor Truck Diesel,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tuesday,17,afternoon,Tuesday afternoon,week,week afternoon,Passing Too Closely
7,2021-12-14 08:17:00,BRONX,10475.0,40.868160,-73.831480,0,0,344 BAYCHESTER AVENUE,2,0,0,0,0,0,2,0,Unspecified,Unspecified,0,0,0,4486660,Sedan,Sedan,0,0,0,2,0.0,0.0,1.0,NaN,NaN,NaN,Motorist,NaN,0.0,Tuesday,8,morning,Tuesday morning,week,week morning,Unspecified
8,2021-12-14 21:10:00,BROOKLYN,11207.0,40.671720,-73.897100,0,0,2047 PITKIN AVENUE,0,0,0,0,0,0,0,0,Driver Inexperience,Unspecified,0,0,0,4487074,Sedan,0,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tuesday,21,night,Tuesday night,week,week night,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044336,2023-07-03 18:05:00,0,0.0,40.866806,-73.931010,RIVERSIDE DRIVE,0,0,0,0,0,0,0,0,0,0,Turning Improperly,Unspecified,0,0,0,4648110,Sedan,Sedan,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Monday,18,afternoon,Monday afternoon,week,week afternoon,Turning Improperly
2044337,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,EAST 174 STREET,WEBSTER AVENUE,0,1,0,1,0,0,0,0,0,Unspecified,0,0,0,0,4648117,Sedan,0,0,0,0,1,1.0,0.0,0.0,NaN,NaN,NaN,Pedestrian,NaN,0.0,Saturday,21,night,Saturday night,weekend,weekend night,Unspecified
2044338,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,WEST STREET,LIBERTY STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,0,0,0,0,4648366,Taxi,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Sunday,17,afternoon,Sunday afternoon,weekend,weekend afternoon,Driver Inattention/Distraction
2044339,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,110 AVENUE,157 STREET,0,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,0,0,0,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Saturday,13,noon,Saturday noon,weekend,weekend noon,Driver Inattention/Distraction


In [10]:
print(ny.LATITUDE.describe())
ny=ny[(ny.LATITUDE>40)&(ny.LATITUDE<42)]
print(ny.LATITUDE.describe())

count   1808390.000000
mean         40.724159
std           0.079594
min          30.784180
25%          40.668316
50%          40.721180
75%          40.769750
max          43.344444
Name: LATITUDE, dtype: float64
count   1808384.000000
mean         40.724163
std           0.079075
min          40.498949
25%          40.668316
50%          40.721180
75%          40.769750
max          41.916610
Name: LATITUDE, dtype: float64


In [15]:
print(ny.LONGITUDE.describe())
ny=ny[(ny.LONGITUDE>-75)&(ny.LONGITUDE<-70)]
print(ny.LONGITUDE.describe())

count   1808262.000000
mean        -73.920130
std           0.092768
min         -79.619550
25%         -73.974945
50%         -73.927500
75%         -73.867230
max         -47.209625
Name: LONGITUDE, dtype: float64
count   1808258.000000
mean        -73.920171
std           0.086049
min         -74.742000
25%         -73.974945
50%         -73.927500
75%         -73.867230
max         -73.663010
Name: LONGITUDE, dtype: float64


In [16]:
len(ny)

1808258

In [21]:
ny['NUMBER OF CYCLIST INJURED'].describe()

count   1808258.000000
mean          0.027818
std           0.166350
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           4.000000
Name: NUMBER OF CYCLIST INJURED, dtype: float64

In [33]:
pedestrians=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF PEDESTRIANS INJURED']]
pedestrians.rename(columns={'NUMBER OF PEDESTRIANS INJURED':'injured'},inplace=True)
pedestrians['typeinjured']='Pedestrian'

cyclist=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF CYCLIST INJURED']]
cyclist.rename(columns={'NUMBER OF CYCLIST INJURED':'injured'},inplace=True)
cyclist['typeinjured']='Cyclist'

motorists=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF MOTORIST INJURED']]
motorists.rename(columns={'NUMBER OF MOTORIST INJURED':'injured'},inplace=True)
motorists['typeinjured']='Motorist'

injured=pd.concat([pedestrians,cyclist,motorists],ignore_index=True)
#injured.sort_values('ZIP CODE',inplace=True)
injured

C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/74425468.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedestrians['typeinjured']='Pedestrian'
C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/74425468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cyclist['typeinjured']='Cyclist'
C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/74425468.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,injured,typeinjured
0,BROOKLYN,11208.000000,40.667202,-73.866500,0,Pedestrian
1,BROOKLYN,11233.000000,40.683304,-73.917274,0,Pedestrian
2,0,0.000000,40.709183,-73.956825,0,Pedestrian
3,BRONX,10475.000000,40.868160,-73.831480,0,Pedestrian
4,BROOKLYN,11207.000000,40.671720,-73.897100,0,Pedestrian
...,...,...,...,...,...,...
5424769,0,0.000000,40.866806,-73.931010,0,Motorist
5424770,BRONX,10457.000000,40.844177,-73.902920,0,Motorist
5424771,MANHATTAN,10006.000000,40.711033,-74.014540,0,Motorist
5424772,QUEENS,11433.000000,40.691580,-73.793190,0,Motorist


In [37]:
injured['ZIP CODE'].nunique()

426

In [40]:
injured2=pd.DataFrame(injured.groupby(['ZIP CODE','typeinjured']).agg({
    'injured': 'sum'
}).reset_index())
injured2

,ZIP CODE,typeinjured,injured
0,0.000000,Cyclist,9192
1,0.000000,Motorist,121207
2,0.000000,Pedestrian,16604
3,10000.000000,Cyclist,68
4,10000.000000,Motorist,70
...,...,...,...
1273,11695,Motorist,4
1274,11695,Pedestrian,0
1275,11697,Cyclist,4
1276,11697,Motorist,10


In [44]:
# Agrupa por 'ZIP CODE' y 'typeinjured' y calcula la suma para cada grupo
grouped_data = injured2.groupby(['ZIP CODE', 'typeinjured'])['injured'].sum().reset_index(name='sum_injured')

# Calcula la suma total de heridos por 'ZIP CODE'
total_by_zip = grouped_data.groupby('ZIP CODE')['sum_injured'].sum().reset_index(name='Total')

# Fusiona el DataFrame original con las sumas totales
result_injured2 = pd.merge(injured2, total_by_zip, on='ZIP CODE', how='left')

# Calcula la columna de porcentaje para cada tipo de herido
for injury_type in ['Cyclist', 'Motorist', 'Pedestrian']:
    result_injured2[f'{injury_type}_percentage'] = (result_injured2['injured'] / result_injured2['Total']) * 100

# Imprime el DataFrame resultante
result_injured2


,ZIP CODE,typeinjured,injured,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage
0,0.000000,Cyclist,9192,147003,6.252934,6.252934,6.252934
1,0.000000,Motorist,121207,147003,82.452059,82.452059,82.452059
2,0.000000,Pedestrian,16604,147003,11.295008,11.295008,11.295008
3,10000.000000,Cyclist,68,153,44.444444,44.444444,44.444444
4,10000.000000,Motorist,70,153,45.751634,45.751634,45.751634
...,...,...,...,...,...,...,...
1273,11695,Motorist,4,4,100.000000,100.000000,100.000000
1274,11695,Pedestrian,0,4,0.000000,0.000000,0.000000
1275,11697,Cyclist,4,14,28.571429,28.571429,28.571429
1276,11697,Motorist,10,14,71.428571,71.428571,71.428571


In [46]:
result_injured2.loc[result_injured2.typeinjured=='Cyclist',['Motorist_percentage','Pedestrian_percentage']]=np.nan
result_injured2.loc[result_injured2.typeinjured=='Motorist',['Cyclist_percentage','Pedestrian_percentage']]=np.nan
result_injured2.loc[result_injured2.typeinjured=='Pedestrian',['Motorist_percentage','Cyclist_percentage']]=np.nan
result_injured2['percentage']=np.nan
result_injured2.loc[~result_injured2.Cyclist_percentage.isna(),'percentage']=result_injured2.Cyclist_percentage
result_injured2.loc[~result_injured2.Motorist_percentage.isna(),'percentage']=result_injured2.Motorist_percentage
result_injured2.loc[~result_injured2.Pedestrian_percentage.isna(),'percentage']=result_injured2.Pedestrian_percentage
result_injured2

,ZIP CODE,typeinjured,injured,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage,percentage
0,0.000000,Cyclist,9192,147003,6.252934,NaN,NaN,6.252934
1,0.000000,Motorist,121207,147003,NaN,82.452059,NaN,82.452059
2,0.000000,Pedestrian,16604,147003,NaN,NaN,11.295008,11.295008
3,10000.000000,Cyclist,68,153,44.444444,NaN,NaN,44.444444
4,10000.000000,Motorist,70,153,NaN,45.751634,NaN,45.751634
...,...,...,...,...,...,...,...,...
1273,11695,Motorist,4,4,NaN,100.000000,NaN,100.000000
1274,11695,Pedestrian,0,4,NaN,NaN,0.000000,0.000000
1275,11697,Cyclist,4,14,28.571429,NaN,NaN,28.571429
1276,11697,Motorist,10,14,NaN,71.428571,NaN,71.428571


In [48]:
injured3=injured.drop_duplicates('ZIP CODE')

In [49]:
result_injured2['BOROUGH']=result_injured2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['BOROUGH'])
result_injured2['LATITUDE']=result_injured2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['LATITUDE'])
result_injured2['LONGITUDE']=result_injured2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['LONGITUDE'])
result_injured2

,ZIP CODE,typeinjured,injured,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage,percentage,BOROUGH,LATITUDE,LONGITUDE
0,0.000000,Cyclist,9192,147003,6.252934,NaN,NaN,6.252934,0,40.709183,-73.956825
1,0.000000,Motorist,121207,147003,NaN,82.452059,NaN,82.452059,0,40.709183,-73.956825
2,0.000000,Pedestrian,16604,147003,NaN,NaN,11.295008,11.295008,0,40.709183,-73.956825
3,10000.000000,Cyclist,68,153,44.444444,NaN,NaN,44.444444,MANHATTAN,40.780550,-73.961690
4,10000.000000,Motorist,70,153,NaN,45.751634,NaN,45.751634,MANHATTAN,40.780550,-73.961690
...,...,...,...,...,...,...,...,...,...,...,...
1273,11695,Motorist,4,4,NaN,100.000000,NaN,100.000000,QUEENS,40.566981,-73.882423
1274,11695,Pedestrian,0,4,NaN,NaN,0.000000,0.000000,QUEENS,40.566981,-73.882423
1275,11697,Cyclist,4,14,28.571429,NaN,NaN,28.571429,QUEENS,40.560825,-73.913219
1276,11697,Motorist,10,14,NaN,71.428571,NaN,71.428571,QUEENS,40.560825,-73.913219


In [34]:
pedestrians=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF PEDESTRIANS KILLED']]
pedestrians.rename(columns={'NUMBER OF PEDESTRIANS KILLED':'KILLED'},inplace=True)
pedestrians['typeKILLED']='Pedestrian'

cyclist=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF CYCLIST KILLED']]
cyclist.rename(columns={'NUMBER OF CYCLIST KILLED':'KILLED'},inplace=True)
cyclist['typeKILLED']='Cyclist'

motorists=ny[['BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF MOTORIST KILLED']]
motorists.rename(columns={'NUMBER OF MOTORIST KILLED':'KILLED'},inplace=True)
motorists['typeKILLED']='Motorist'

KILLED=pd.concat([pedestrians,cyclist,motorists],ignore_index=True)
#KILLED.sort_values('ZIP CODE',inplace=True)
KILLED

C:\Users\52269198\Anaconda3\envs\strong\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/1012911524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pedestrians['typeKILLED']='Pedestrian'
C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/1012911524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,KILLED,typeKILLED
0,BROOKLYN,11208.000000,40.667202,-73.866500,0,Pedestrian
1,BROOKLYN,11233.000000,40.683304,-73.917274,0,Pedestrian
2,0,0.000000,40.709183,-73.956825,0,Pedestrian
3,BRONX,10475.000000,40.868160,-73.831480,0,Pedestrian
4,BROOKLYN,11207.000000,40.671720,-73.897100,0,Pedestrian
...,...,...,...,...,...,...
5424769,0,0.000000,40.866806,-73.931010,0,Motorist
5424770,BRONX,10457.000000,40.844177,-73.902920,0,Motorist
5424771,MANHATTAN,10006.000000,40.711033,-74.014540,0,Motorist
5424772,QUEENS,11433.000000,40.691580,-73.793190,0,Motorist


In [50]:
killed2=pd.DataFrame(KILLED.groupby(['ZIP CODE','typeKILLED']).agg({
    'KILLED': 'sum'
}).reset_index())
killed2

,ZIP CODE,typeKILLED,KILLED
0,0.000000,Cyclist,48
1,0.000000,Motorist,413
2,0.000000,Pedestrian,353
3,10000.000000,Cyclist,0
4,10000.000000,Motorist,0
...,...,...,...
1273,11695,Motorist,0
1274,11695,Pedestrian,0
1275,11697,Cyclist,0
1276,11697,Motorist,0


In [53]:
# Agrupa por 'ZIP CODE' y 'typekilled' y calcula la suma para cada grupo
grouped_data = killed2.groupby(['ZIP CODE', 'typeKILLED'])['KILLED'].sum().reset_index(name='sum_killed')

# Calcula la suma total de heridos por 'ZIP CODE'
total_by_zip = grouped_data.groupby('ZIP CODE')['sum_killed'].sum().reset_index(name='Total')

# Fusiona el DataFrame original con las sumas totales
result_killed2 = pd.merge(killed2, total_by_zip, on='ZIP CODE', how='left')

# Calcula la columna de porcentaje para cada tipo de herido
for injury_type in ['Cyclist', 'Motorist', 'Pedestrian']:
    result_killed2[f'{injury_type}_percentage'] = (result_killed2['KILLED'] / result_killed2['Total']) * 100

# Imprime el DataFrame resultante
result_killed2

,ZIP CODE,typeKILLED,KILLED,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage
0,0.000000,Cyclist,48,814,5.896806,5.896806,5.896806
1,0.000000,Motorist,413,814,50.737101,50.737101,50.737101
2,0.000000,Pedestrian,353,814,43.366093,43.366093,43.366093
3,10000.000000,Cyclist,0,0,NaN,NaN,NaN
4,10000.000000,Motorist,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1273,11695,Motorist,0,0,NaN,NaN,NaN
1274,11695,Pedestrian,0,0,NaN,NaN,NaN
1275,11697,Cyclist,0,0,NaN,NaN,NaN
1276,11697,Motorist,0,0,NaN,NaN,NaN


In [56]:
result_killed2.loc[result_killed2.typeKILLED=='Cyclist',['Motorist_percentage','Pedestrian_percentage']]=np.nan
result_killed2.loc[result_killed2.typeKILLED=='Motorist',['Cyclist_percentage','Pedestrian_percentage']]=np.nan
result_killed2.loc[result_killed2.typeKILLED=='Pedestrian',['Motorist_percentage','Cyclist_percentage']]=np.nan
result_killed2['percentage']=np.nan
result_killed2.loc[~result_killed2.Cyclist_percentage.isna(),'percentage']=result_killed2.Cyclist_percentage
result_killed2.loc[~result_killed2.Motorist_percentage.isna(),'percentage']=result_killed2.Motorist_percentage
result_killed2.loc[~result_killed2.Pedestrian_percentage.isna(),'percentage']=result_killed2.Pedestrian_percentage
result_killed2

,ZIP CODE,typeKILLED,KILLED,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage,percentage
0,0.000000,Cyclist,48,814,5.896806,NaN,NaN,5.896806
1,0.000000,Motorist,413,814,NaN,50.737101,NaN,50.737101
2,0.000000,Pedestrian,353,814,NaN,NaN,43.366093,43.366093
3,10000.000000,Cyclist,0,0,NaN,NaN,NaN,NaN
4,10000.000000,Motorist,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1273,11695,Motorist,0,0,NaN,NaN,NaN,NaN
1274,11695,Pedestrian,0,0,NaN,NaN,NaN,NaN
1275,11697,Cyclist,0,0,NaN,NaN,NaN,NaN
1276,11697,Motorist,0,0,NaN,NaN,NaN,NaN


In [58]:
result_killed2['BOROUGH']=result_killed2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['BOROUGH'])
result_killed2['LATITUDE']=result_killed2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['LATITUDE'])
result_killed2['LONGITUDE']=result_killed2['ZIP CODE'].map(injured3.set_index('ZIP CODE')['LONGITUDE'])
result_killed2

,ZIP CODE,typeKILLED,KILLED,Total,Cyclist_percentage,Motorist_percentage,Pedestrian_percentage,percentage,BOROUGH,LATITUDE,LONGITUDE
0,0.000000,Cyclist,48,814,5.896806,NaN,NaN,5.896806,0,40.709183,-73.956825
1,0.000000,Motorist,413,814,NaN,50.737101,NaN,50.737101,0,40.709183,-73.956825
2,0.000000,Pedestrian,353,814,NaN,NaN,43.366093,43.366093,0,40.709183,-73.956825
3,10000.000000,Cyclist,0,0,NaN,NaN,NaN,NaN,MANHATTAN,40.780550,-73.961690
4,10000.000000,Motorist,0,0,NaN,NaN,NaN,NaN,MANHATTAN,40.780550,-73.961690
...,...,...,...,...,...,...,...,...,...,...,...
1273,11695,Motorist,0,0,NaN,NaN,NaN,NaN,QUEENS,40.566981,-73.882423
1274,11695,Pedestrian,0,0,NaN,NaN,NaN,NaN,QUEENS,40.566981,-73.882423
1275,11697,Cyclist,0,0,NaN,NaN,NaN,NaN,QUEENS,40.560825,-73.913219
1276,11697,Motorist,0,0,NaN,NaN,NaN,NaN,QUEENS,40.560825,-73.913219


In [66]:
result_killed2=result_killed2[result_killed2['ZIP CODE']!=0]

In [68]:
result_killed2.percentage.describe()

count   930.000000
mean     33.333333
std      34.525477
min       0.000000
25%       0.000000
50%      25.000000
75%      57.142857
max     100.000000
Name: percentage, dtype: float64

In [75]:
result_killed2.Total.describe()

count   1275.000000
mean       4.110588
std        5.540022
min        0.000000
25%        0.000000
50%        2.000000
75%        6.000000
max       55.000000
Name: Total, dtype: float64

In [76]:
result_killed2['TOP percentages']=np.nan
result_killed2.loc[(result_killed2.percentage>57)&(result_killed2.KILLED>6),'TOP percentages']='Top'
result_killed2['TOP percentages'].value_counts()

C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/3183745057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_killed2['TOP percentages']=np.nan
C:\Users\52269198\Anaconda3\envs\strong\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Top    40
Name: TOP percentages, dtype: int64

In [70]:
result_injured2=result_injured2[result_injured2['ZIP CODE']!=0]

In [77]:
result_injured2.percentage.describe()

count   1227.000000
mean      33.333333
std       26.914339
min        0.000000
25%       11.420529
50%       24.550360
75%       55.734756
max      100.000000
Name: percentage, dtype: float64

In [79]:
result_injured2.Total.describe()

count    1275.000000
mean      944.195294
std      1225.526283
min         0.000000
25%       166.000000
50%       512.000000
75%      1226.000000
max     10143.000000
Name: Total, dtype: float64

In [80]:
result_injured2['TOP percentages']=np.nan
result_injured2.loc[(result_injured2.percentage>56)&(result_injured2.injured>1226),'TOP percentages']='Top'
result_injured2['TOP percentages'].value_counts()

C:\Users\52269198\AppData\Local\Temp/ipykernel_12684/2208550971.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_injured2['TOP percentages']=np.nan
C:\Users\52269198\Anaconda3\envs\strong\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Top    64
Name: TOP percentages, dtype: int64

In [81]:
result_injured2.to_csv(path+"injured_piecharts.csv",index=False)
result_killed2.to_csv(path+"killed_piecharts.csv",index=False)

In [17]:
ny.to_csv(path+"Motor_Vehicle_Collisions_-_Crashes_20231122.csv",index=False)